<a href="https://colab.research.google.com/github/Sanarazaaa/Neural_Oscillation-and-Burst-Firing-Simulation/blob/main/Neural_Oscillation_and_Burst_Firing_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy.signal import find_peaks, hilbert
from scipy.fftpack import fft, fftfreq

# Simulation parameters
dt = 0.1  # Time step (ms)
T = 1000  # Total time (ms)
time = np.arange(0, T, dt)

# Neuron parameters
V_rest = -65  # Resting membrane potential (mV)
V_th = -50    # Threshold potential (mV)
V_reset = -65 # Reset potential after spike
tau = 10      # Membrane time constant (ms)

# External oscillatory input (theta: 5 Hz, gamma: 40 Hz)
theta_freq = 5
gamma_freq = 40
I_theta = np.sin(2 * np.pi * theta_freq * time / 1000)
I_gamma = np.sin(2 * np.pi * gamma_freq * time / 1000)

# Function to simulate neuron firing
def simulate_neuron(GNaP, GKDR, Gh):
    V = V_rest
    spikes = []
    V_trace = []

    for t in range(len(time)):
        I_input = 0.2 * I_theta[t] + 0.1 * I_gamma[t]  # Combined oscillatory input

        # Incorporate hyperpolarization-activated current (Ih)
        Ih_effect = Gh * (V_rest - V)

        # Neuron dynamics
        dV = (V_rest - V + GNaP * I_input - GKDR * (V - V_rest) + Ih_effect) / tau
        V += dV * dt

        # Spike detection
        if V >= V_th:
            V = V_reset
            spikes.append(time[t])

        V_trace.append(V)

    # Burst Detection (Find consecutive spikes within 30 ms)
    spikes = np.array(spikes)
    burst_indices = np.where(np.diff(spikes) < 30)[0]  # Bursts
    single_spikes = np.setdiff1d(np.arange(len(spikes)), burst_indices)  # Single spikes

    # Calculate Burst Ratio
    burst_ratio = len(burst_indices) / len(spikes) if len(spikes) > 0 else 0

    # Plot Membrane Potential
    plt.figure(figsize=(10, 5))
    plt.plot(time, V_trace, label="Membrane Potential (mV)")
    plt.scatter(spikes[single_spikes], [V_th]*len(single_spikes), color='red', label="Single Spikes", marker='o')
    plt.scatter(spikes[burst_indices], [V_th]*len(burst_indices), color='blue', label="Burst Spikes", marker='o')
    plt.xlabel("Time (ms)")
    plt.ylabel("Membrane Potential (mV)")
    plt.title(f"Neuron Spiking (GNaP={GNaP}, GKDR={GKDR}, Gh={Gh}) - Burst Ratio: {burst_ratio:.2f}")
    plt.legend()
    plt.show()

    # Resonance Analysis with FFT
    fft_vals = np.abs(fft(V_trace))[:len(V_trace)//2]  # Compute FFT
    fft_freqs = fftfreq(len(V_trace), d=dt)[:len(V_trace)//2]  # Frequency axis

    # Plot Frequency Spectrum
    plt.figure(figsize=(10, 4))
    plt.plot(fft_freqs, fft_vals, color='purple')
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Power")
    plt.title("Neural Resonance Analysis (FFT)")
    plt.xlim(0, 100)  # Focus on relevant frequencies
    plt.show()

    # Phase Locking Analysis: Modified version
    if len(spikes) > 0:  # Only create phase plot if there are spikes
        # Get phases of theta oscillation (primary rhythm for phase-locking)
        theta_phase = np.angle(hilbert(I_theta))

        # Convert spike times to indices
        spike_indices = np.array([int(s/dt) for s in spikes])
        # Ensure indices are within bounds
        spike_indices = spike_indices[spike_indices < len(theta_phase)]

        # Get spike phases (convert to degrees for better visualization)
        spike_phases = np.rad2deg(theta_phase[spike_indices]) % 360

        plt.figure(figsize=(8, 6))
        # Create circular histogram (rose plot)
        bins = np.linspace(0, 360, 37)  # 36 bins of 10 degrees each
        plt.hist(spike_phases, bins=bins, color='darkblue', alpha=0.7)
        plt.xlabel("Phase (degrees)")
        plt.ylabel("Spike Count")
        plt.title("Phase Locking to Theta Rhythm")

        # Add reference lines
        plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
        plt.axvline(x=180, color='gray', linestyle='--', alpha=0.5)
        plt.axvline(x=360, color='gray', linestyle='--', alpha=0.5)

        # Set x-axis limits and ticks
        plt.xlim(0, 360)
        plt.xticks(np.arange(0, 361, 90))
        plt.show()

# Interactive Sliders
GNaP_slider = widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=0.5, description="GNaP")
GKDR_slider = widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=0.3, description="GKDR")
Gh_slider = widgets.FloatSlider(min=0.0, max=0.5, step=0.05, value=0.1, description="Gh")  # Hyperpolarization Current

interactive_plot = widgets.interactive(simulate_neuron, GNaP=GNaP_slider, GKDR=GKDR_slider, Gh=Gh_slider)
display(interactive_plot)

interactive(children=(FloatSlider(value=0.5, description='GNaP', max=1.0, min=0.1), FloatSlider(value=0.3, des…